In [1]:
import torch
# from diffusers import StableDiffusionInpaintingPipeline
from torchvision.utils import save_image
from torchvision import transforms
from PIL import Image
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
import sys
sys.path.insert(0, "Image_Inpainting")
from data import InpaintingDataset
from evaluate_models import evaluate_model
from diffusers import StableDiffusionInpaintPipeline, DiffusionPipeline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

2025-06-10 16:19:14.448558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 16:19:14.448612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 16:19:14.449892: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 16:19:14.456016: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load pretrained inpainting model
# pipe = StableDiffusionInpaintPipeline.from_pretrained(
#     # "stabilityai/stable-diffusion-2-inpainting",
#     "runwayml/stable-diffusion-inpainting",
#     revision="fp16",
#     torch_dtype=torch.float16,
#     # low_cpu_mem_usage=True,
#     # use_safetensors=True
# ).to(device)
# pipe.safety_checker = None  # optional: disable NSFW filter
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-inpainting",
    torch_dtype=torch.float16
).to(device)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [9]:
def run_inpainting(val_loader, save_dir="celeba_inpainted_outputs", prompt="", stop_at=5):
    os.makedirs(save_dir, exist_ok=True)
    cnt = 0
    
    for i, (masked_image, img, mask) in enumerate(val_loader):
        cnt += 1
        # print(masked_image.shape, img.shape, mask.shape)
        # result = pipe(prompt=prompt, image=masked_image, mask_image=mask).images[0]
        result = pipe(prompt=prompt, image=img, mask_image=mask).images[0]

        result.save(os.path.join(save_dir, f"inpainted_{i:04d}.png"))
        save_image(masked_image[0], os.path.join(save_dir, f"masked_{i:04d}.png"))
        save_image(img[0], os.path.join(save_dir, f"original_{i:04d}.png"))
        # img.save(os.path.join(save_dir, f"original_{i:04d}.png"))

        if i % 10 == 0:
            print(f"[{i}] Inpainted image saved.")

        if cnt == 5:
            break

In [10]:
# dataset = InpaintingDataset(root="data/celeba_hq_256")
dataset = InpaintingDataset(root="data/cityscapes/val/img")
# dataset = dataset[:10]
print(len(dataset))
loader = DataLoader(dataset, batch_size=1, 
                        pin_memory=True, shuffle=False, persistent_workers=True, 
                        num_workers=24)


[INFO] Found 500 images for 'celeba' in data/cityscapes/val/img (recursively, single class)
500


In [11]:
run_inpainting(loader, save_dir="cityscapes_inpainted_outputs")

  0%|          | 0/50 [00:00<?, ?it/s]

[0] Inpainted image saved.


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]